# Excercise 6
## NLP with Pytorch 🔥

In [120]:
!pip install contractions

In [121]:
import nltk
nltk.download('wordnet')
nltk.download('punkt')
import re

from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import pad_sequence
import re
import contractions



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [122]:
import numpy as np
import keras
import pandas as pd
import matplotlib.pyplot as plt

In [123]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler


## 5.1 Predict rating of a movie using Keras

**Exercise:** Use keras framework to predict rating.

In [124]:
dataTraining = pd.read_csv('https://github.com/sergiomora03/AdvancedTopicsAnalytics/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)

En este paso, hicimos a concatenación de los campos plot, title y genres. Mejoró el resultado del modelo al final.

In [125]:
X = dataTraining.drop(columns=['rating'])
X = dataTraining['title'].str.cat(dataTraining['plot'], sep=', ').str.cat(dataTraining['genres'], sep=', ')
y = (dataTraining['rating'] >= dataTraining['rating'].mean()).astype(int)

In [126]:
X

,title
3107,"Most, most is the story of a single father who..."
900,"How to Be a Serial Killer, a serial killer dec..."
6724,"A Woman's Face, in sweden , a female blackmai..."
4704,"Executive Suite, in a friday afternoon in new ..."
2582,"Narrow Margin, in los angeles , the editor of..."
...,...
8417,"Our Family Wedding, "" our marriage , their we..."
1592,"Conan the Destroyer, the wandering barbarian ,..."
1723,"Kismet, like a tale spun by scheherazade , ki..."
7605,"The Secret of NIMH, mrs . brisby , a widowed..."


In [127]:
dataTraining

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6
...,...,...,...,...,...
8417,2010,Our Family Wedding,""" our marriage , their wedding . "" it ' s l...","['Comedy', 'Romance']",4.9
1592,1984,Conan the Destroyer,"the wandering barbarian , conan , alongside ...","['Action', 'Adventure', 'Fantasy']",5.8
1723,1955,Kismet,"like a tale spun by scheherazade , kismet fol...","['Adventure', 'Musical', 'Fantasy', 'Comedy', ...",6.4
7605,1982,The Secret of NIMH,"mrs . brisby , a widowed mouse , lives in a...","['Animation', 'Adventure', 'Drama', 'Family', ...",7.6


In [128]:
y

,rating
3107,1
900,0
6724,1
4704,1
2582,1
...,...
8417,0
1592,0
1723,0
7605,1


## Data Precosessing

- Remove stopwords
- Lowercase
- split the text in words
- pad_sequences

En este paso hacemos el preprocesamiento de los datos. Se deja todo en minúscula, se eliminan signos de puntuación, se eliminan espacios en blanco adicionales, y por último aplicamos lematización.

In [129]:
def preprocess_text(text):

    text = text.lower() # Convierte a minúsculas
    text = re.sub(r'[^\w\s]', '', text) # Elimina caracteres especiales
    text = re.sub(r'\s+', ' ', text) # Elimina espacios en blanco adicionales
    text = re.sub('-',' ',text.lower())   # reemplazamos `word-word` as `word word`
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = nltk.pad_sequence(tokens, 2)
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
# Aplica preprocesamiento a la columna title
X = X.apply(preprocess_text)

Así se visualiza nuestra feature, ya con el preprocesamiento.

In [130]:
X

,title
3107,story single father take eight year old son wo...
900,serial killer serial killer decides teach secr...
6724,woman face sweden female blackmailer disfiguri...
4704,executive suite friday afternoon new york pres...
2582,narrow margin los angeles editor publishing ho...
...,...
8417,family wedding marriage wedding lesson number ...
1592,conan destroyer wandering barbarian conan alon...
1723,kismet like tale spun scheherazade kismet foll...
7605,secret nimh mr brisby widowed mouse life cinde...


## Build Model

Create a neural network to predict the rating of a movie, calculate the testing set accuracy.

1. Separamos los datos en training y test. Vamos a utilizar una proporción de 80 para training y 20 para tests.

In [131]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch.optim import Adam

Dividimos la data en entrenamiento y test. Con 80% para entrenamiento y 20% para test.

In [132]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=256)

Creamos el vocabilario a partir de los tokens.

In [133]:

from sklearn.feature_extraction.text import CountVectorizer

Vectorizamos y transformamos a matriz tanto el conjunto de entrenamiento como el conjunto de test.

In [134]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

Dado que pytorch trabaja con tensores, es necesario que transformemos nuestros datos a tensores. Sin este paso, pythorch no podrá procesar los datos de manera eficiente.

In [135]:

X_train_tensor = torch.tensor(X_train_vec.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test_vec.toarray(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

En esta red neuronal, tenemos 3 capas, 2 ocultas con ReLu y una de salida (La predicción).

In [136]:

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Instanciamos el modelo y definimos una función de pérdida Cross Entropy que nos ayudará a evaluar el rendimiento del modelo.

In [137]:
model = Net(X_train_tensor.shape[1])
optimizer = Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

Entrenamos la red con 15 epocas, utilizamos un optimizer.step() que nos va actualizando los parámetros del modelo para minimizar la pérdida.

In [138]:
epochs = 15
batch_size = 32

for epoch in range(epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        X_batch = X_train_tensor[i:i+batch_size]
        y_batch = y_train_tensor[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    accuracy = accuracy_score(y_test_tensor, predicted)

print(f"Test Accuracy: {accuracy}")

Epoch 1/15, Loss: 0.8897997736930847
Epoch 2/15, Loss: 0.08452267199754715
Epoch 3/15, Loss: 0.002076247474178672
Epoch 4/15, Loss: 7.641185220563784e-05
Epoch 5/15, Loss: 4.1332110413350165e-05
Epoch 6/15, Loss: 2.2986394469626248e-05
Epoch 7/15, Loss: 1.4553013897966594e-05
Epoch 8/15, Loss: 8.950458322942723e-06
Epoch 9/15, Loss: 6.069660685170675e-06
Epoch 10/15, Loss: 4.271626949048368e-06
Epoch 11/15, Loss: 3.139156433462631e-06
Epoch 12/15, Loss: 2.394107468717266e-06
Epoch 13/15, Loss: 1.847736371018982e-06
Epoch 14/15, Loss: 1.4702430917168385e-06
Epoch 15/15, Loss: 1.1920897122763563e-06
Test Accuracy: 0.6345788473717543
